In [1]:
import sys
import re
from nltk import sent_tokenize, word_tokenize, pos_tag, ne_chunk
import nltk.data
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
import pickle
from collections.abc import Iterable
import random
from lxml import etree
import gzip

from nltk.tag import ClassifierBasedTagger
from nltk.chunk import ChunkParserI
import unidecode
import string
from nltk.stem.snowball import SnowballStemmer
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn import metrics
import random
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import matplotlib 
matplotlib.use('Agg')
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2
from unwiki import unwiki
from ner.chunker import NamedEntityChunker
import ner

In [3]:
%%script echo opens the latest wikipedia defs
# The results for the search for definition (currently just Wikipedia)
with open('/media/hd1/wikipedia/wiki_definitions_improved.txt', 'r') as wiki_f:
    wikilines = wiki_f.readlines()

opens the latest wikipedia defs


In [4]:
%%script echo Save and create xml zipped file
root = etree.Element('root')

for n, line in enumerate(wikilines):
    title, section, defin = line.split('-#-%-')
    defin = unwiki.loads(eval(defin)).replace('\n', ' ')
    def_tree = etree.Element('definition')
    stmnt = etree.SubElement(def_tree, 'stmnt')
    stmnt.text = unidecode.unidecode(defin)
    dfndum = etree.SubElement(def_tree, 'dfndum')
    dfndum.text = title.strip()
    if stmnt.text != '':
        root.append(def_tree)
    #if n > 4976:
    #    break
#print(etree.tostring(root, pretty_print=True).decode('utf8')) 

with gzip.open('/media/hd1/wikipedia/wiki_definitions_improved.xml.gz', 'wb') as wiki_fobj:
    wiki_fobj.write(etree.tostring(root, pretty_print=True))
    #print(etree.tostring(root, pretty_print=True).decode('utf8'))

Save and create xml zipped file


In [5]:
def split_fields(elem):
    title = elem.find('.//dfndum').text 
    section = elem.get('name')
    defin = elem.find('.//stmnt').text
    return (title, section, defin)

wiki = []
with gzip.open('/media/hd1/wikipedia/wiki_definitions_improved.xml.gz', 'r') as xml_fobj:
    def_xml = etree.parse(xml_fobj)
    for art in def_xml.findall('definition'):
        data = (art.find('.//dfndum').text, '', art.find('.//stmnt').text)
        wiki.append(data)
        
#random.sample(wiki, 1)
        
plmath = []
with gzip.open('/media/hd1/planetmath/datasets/planetmath_definitions.xml.gz', 'r') as xml_fobj:
    def_xml = etree.parse(xml_fobj)
    for art in def_xml.findall('article'):
        plmath.append(split_fields(art))
stacks = []
with gzip.open('/media/hd1/stacks-project/datasets/stacks-definitions.xml.gz', 'r') as xml_fobj:
    def_xml = etree.parse(xml_fobj)
    for art in def_xml.findall('article'):
        try:
            stacks.append(split_fields(art))
        except AttributeError:
            print('The name of the problematic article is: {}'.format(art.attrib['name']))

# Print results
print('Found {} in the wiki dataset.'.format(len(wiki)))
print('Found {} in the stacks dataset.'.format(len(stacks)))
print('Found {} in the PlanetMath dataset.'.format(len(plmath)))

The name of the problematic article is: examples.xml
The name of the problematic article is: coding.xml
The name of the problematic article is: spaces-pushouts.xml
The name of the problematic article is: guide.xml
The name of the problematic article is: moduli.xml
The name of the problematic article is: more-groupoids.xml
The name of the problematic article is: chapters.xml
The name of the problematic article is: sets.xml
The name of the problematic article is: obsolete.xml
The name of the problematic article is: examples-defos.xml
The name of the problematic article is: spaces-more-cohomology.xml
The name of the problematic article is: bibliography.xml
The name of the problematic article is: fdl.xml
The name of the problematic article is: limits.xml
The name of the problematic article is: conventions.xml
The name of the problematic article is: introduction.xml
The name of the problematic article is: quot.xml
The name of the problematic article is: desirables.xml
Found 5314 in the wiki

In [6]:
# Get data and train the Sentence tokenizer
# Uses a standard algorithm (Kiss-Strunk) for unsupervised sentence boundary detection
text = ''
for i in range(550):
    #text += unwiki.loads(eval(wiki[i].split('-#-%-')[2]))
    text += plmath[i][2]
for i in range(50):
    text += stacks[i][2]
for i in range(550):
    try:
        text += wiki[i][2]
    except TypeError:
        print(wiki[i])

trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)
tokenizer = PunktSentenceTokenizer(trainer.get_params())
print(tokenizer._params.abbrev_types)

{'jie', 'axe', 'missions', 'brethren', 'vol', 'eng', 'r.a', 'ex', '62f', 'sgs', 'og', 'a.e', 'spacewalks', 'resp', 'ginebra', 'forever—e.g', 'wings', 'jr', 'cf', '15k', 'u.n', 'vibrations', 'a.k.a', 'eqs', 'i.e', 'al', 'mixture', 'juniper', 'eq', 'etc', 'ton', 'p.h.d', 'pl', 'sow', 'fig', 'z-1', 'n.b', 'i.o', 'j.a', 'scand', 'inc', 's.t', 'az', '2p', 'e.g', 'c.c.c', 'pp', 's^2', 'x+2', 'mr', 'ca', 'hk', 'dr', 'u.s', 'hyp', 'j.w'}


In [7]:
# The cleaning up of the wiki markup so that it looks like normal written english
title, section, defin = plmath[850]
print(title)
print(section)
defin

anti self-dual
15A63-Selfdual.xml


' Let _inline_math_ be a finite-dimensional inner-product space over a field _inline_math_. Let _inline_math_ be an endomorphism, and note that the adjoint endomorphism _inline_math_ is also an endomorphism of _inline_math_. It is therefore possible to add, subtract, and compare _inline_math_ and _inline_math_, and we are able to make the following definitions. An endomorphism _inline_math_ is said to be self-dual (a.k.a. self-adjoint) if _display_math_ By contrast, we say that the endomorphism is anti self-dual if _display_math_  Exactly the same definitions can be made for an endomorphism of a complex vector space with a Hermitian inner product.  All of these definitions have their counterparts in the matrix setting. Let _inline_math_ be the matrix of _inline_math_ relative to an orthogonal basis of _inline_math_. Then _inline_math_ is self-dual if and only if _inline_math_ is a symmetric matrix, and anti self-dual if and only if _inline_math_ is a skew-symmetric matrix.  In the case

In [8]:
# Get the data and POS and NER tags for each definition (LONG TIME)
#def put_pos_ner_tags(defl):
#    '''
#    INPUTS
#    ------
#    defl: list of tuples with the format (title, section, definition)
#    tokenizer: sentence tokenizer, splits paragraphs into sentences and 
#               identifies abbreviations.
#    - Checks if the definiendum is contained in the sentence 
#    - Finds the POS of each word in each sentence
#    '''
#    def_lst = []
#    for i in range(len(defl)):
#        try:
#            #title, section, defin_raw = wiki[i].split('-#-%-')
#            #defin_all = unwiki.loads(eval(defin_raw))
#            title, section, defin_all = defl[i]
#            for d in tokenizer.tokenize(defin_all):
#                if title.lower().strip() in d.lower():
#                    pos_tokens = pos_tag(word_tokenize(d))
#                    def_ner = ner.bio_tag.bio_tagger(title.strip().split(), pos_tokens)
#                    other_ner = [((d[0],d[1]),d[2]) for d in def_ner]
#                    tmp_dict = {'title': title,
#                               'section': section,
#                               'defin': d,
#                               'ner': other_ner}
#                    def_lst.append(tmp_dict)
#        except ValueError:
#            print('parsing error')
#    return def_lst
def_lst = ner.bio_tag.put_pos_ner_tags(stacks, tokenizer)\
         + ner.bio_tag.put_pos_ner_tags(plmath, tokenizer)\
         + ner.bio_tag.put_pos_ner_tags(wiki, tokenizer)

#for i in range(len(wiki)):
#    try:
#        title, section, defin_raw = wiki[i]
#        defin_all = defin_raw
#        for d in tokenizer.tokenize(defin_all):
#            if title.lower().strip() in d.lower():
#                pos_tokens = pos_tag(word_tokenize(d))
#                def_ner = ner.bio_tag.bio_tagger(title.strip().split(), pos_tokens)
#                other_ner = [((d[0],d[1]),d[2]) for d in def_ner]
#                tmp_dict = {'title': title,
#                           'section': section,
#                           'defin': d,
#                           'ner': other_ner}
#                def_lst.append(tmp_dict)
#    except ValueError:
#        print('parsing error')

In [9]:
w = wiki[-12]
w

('Government hacking',
 '',
 "The term hacking is used for actions taken by a group of people named as hackers. Hackers have extensive knowledge in the area of technology, specifically with electronic devices, programs and computer networks. These people differ from crackers because use their knowledge for illegal and unethical purposes. Overall, crackers apply their skills to take advantage of vulnerabilities in software and systems. The hacking action consists of manipulating computer systems or electronic devices in order to remotely control the machine or have access to the data stored there.  Due to the innovation of new technologies, it was necessary to update the cryptographic algorithms. This need has raised the level of complexity of the new techniques created for encrypting the data of the individuals in order to guarantee security in the network. Because of the difficulty of deciphering the data, government agencies have begun to search for other options to conduct criminal 

In [10]:
ner.bio_tag.put_pos_ner_tags([w], tokenizer)

[{'title': 'Government hacking',
  'section': '',
  'defin': 'One such option is the so-called government hacking.',
  'ner': [(('One', 'CD'), 'O'),
   (('such', 'JJ'), 'O'),
   (('option', 'NN'), 'O'),
   (('is', 'VBZ'), 'O'),
   (('the', 'DT'), 'O'),
   (('so-called', 'JJ'), 'O'),
   (('government', 'NN'), 'B-DFNDUM'),
   (('hacking', 'NN'), 'I-DFNDUM'),
   (('.', '.'), 'O')]}]

In [11]:
random.shuffle(def_lst)
training_samples = [d['ner'] for d in def_lst[:int(len(def_lst) * 0.8)]]
test_samples = [d['ner'] for d in def_lst[int(len(def_lst) * 0.8):]]
 
print("#training samples = %s" % len(training_samples) )   # training samples = 55809
print("#test samples = %s" % len(test_samples))            # test samples = 6201

#training samples = 20880
#test samples = 5220


In [12]:
#train the NER Chunking Classifier (TAKES A LONG TIME)
%time chunker = NamedEntityChunker(training_samples)

CPU times: user 23.8 s, sys: 1.81 s, total: 25.6 s
Wall time: 25.6 s


In [13]:
# Evaluate the most common metrics on the test dataset
unpack = lambda l: [(tok, pos, ner) for ((tok, pos), ner) in l]
Tree_lst = [conlltags2tree(unpack(t)) for t in test_samples]
print(chunker.evaluate(Tree_lst))

ChunkParse score:
    IOB Accuracy:  87.7%%
    Precision:     28.5%%
    Recall:        67.8%%
    F-Measure:     40.1%%


In [31]:
print(Tree_lst[3].leaves())
print(chunker.parse(Tree_lst[3].leaves()))

[('It', 'PRP'), ('is', 'VBZ'), ('the', 'DT'), ('“', 'JJ'), ('smallest', 'JJS'), ('”', 'NN'), ('Dynkin', 'NNP'), ('system', 'NN'), ('containing', 'VBG'), ('_inline_math_', 'NN'), ('.', '.')]
(S
  It/PRP
  is/VBZ
  the/DT
  “/JJ
  smallest/JJS
  ”/NN
  (DFNDUM Dynkin/NNP system/NN)
  containing/VBG
  _inline_math_/NN
  ./.)


### Other Scores

Training with an amount of the dataset and evaluating with the rest
* With 80% split of the stacks data:
```
ChunkParse score:
    IOB Accuracy:  86.5%%
    Precision:     29.0%%
    Recall:        60.0%%
    F-Measure:     39.1%%
```

* With 80% of Wiki + Planetmath data:
```
ChunkParse score:
    IOB Accuracy:  88.4%%
    Precision:     26.5%%
    Recall:        66.3%%
    F-Measure:     37.9%%
```

* With 90% of Wiki + Planetmath:
```
ChunkParse score:
    IOB Accuracy:  88.7%%
    Precision:     27.1%%
    Recall:        67.7%%
    F-Measure:     38.7%%
```

* With 90% of the dataset Planetmath data alone:
```
ChunkParse score:
    IOB Accuracy:  86.0%%
    Precision:     24.0%%
    Recall:        67.3%%
    F-Measure:     35.3%%
```
    
* With 80% of the dataset

* 60% of the data

```
ChunkParse score:
    IOB Accuracy:  91.0%%
    Precision:     30.7%%
    Recall:        63.9%%
    F-Measure:     41.5%%
```
    
```
ChunkParse score:
    IOB Accuracy:  90.6%%
    Precision:     32.4%%
    Recall:        68.7%%
    F-Measure:     44.0%%
```

* 90% of the data

```
ChunkParse score:
    IOB Accuracy:  91.2%%
    Precision:     32.0%%
    Recall:        68.0%%
    F-Measure:     43.5%%
```
    


In [15]:
# An example of a user fed definition
print(chunker.parse(pos_tag(word_tokenize("We define a Banach space as a complete vector space."))))

(S
  We/PRP
  define/VBP
  a/DT
  (DFNDUM Banach/NNP space/NN)
  as/IN
  a/DT
  (DFNDUM complete/JJ vector/NN space/NN)
  ./.)


In [16]:
def prepare_for_metrics(int_range, chunker_fn, data_set = test_samples, print_output=False):
    '''
    `int_range` is an integer range
    NEEDS A TEST_SAMPLES VARIABLE CREATED WHEN SPLITTING THE 
    TRAINING AND TESTING DATA
    Returns two vectors ready to be used in the 
    metrics classification function
    '''
    if isinstance(int_range, int):
        int_range = [int_range]
    y_true = []
    y_pred = []
    for i in int_range:
        sample = data_set[i]
        sm = [s[0] for s in sample]
        y_true_tmp = [s[1] for s in sample]
        predicted = [v[2] for v in tree2conlltags(chunker_fn.parse(sm))]
        y_true += y_true_tmp
        y_pred += predicted
        if print_output:
            for k,s in enumerate(sm):
                print('{:15} {:>10}  {:>10}'.format(s[0], y_true_tmp[k], predicted[k]))
    return y_true, y_pred

In [17]:
OO = prepare_for_metrics(19, chunker, data_set=test_samples, print_output=True)

Saltwater                O    B-DFNDUM
pearls            B-DFNDUM    I-DFNDUM
can                      O           O
grow                     O           O
in                       O           O
several                  O           O
species                  O           O
of                       O           O
marine                   O           O
pearl             B-DFNDUM    B-DFNDUM
oysters                  O    I-DFNDUM
in                       O           O
the                      O           O
family                   O    B-DFNDUM
Pteriidae                O    I-DFNDUM
.                        O           O


In [22]:
y_true, predicted = prepare_for_metrics(range(len(test_samples)), chunker)
print(metrics.classification_report(y_true, predicted))

              precision    recall  f1-score   support

    B-DFNDUM       0.32      0.77      0.45      5923
    I-DFNDUM       0.30      0.85      0.45      4434
           O       0.99      0.88      0.93    150236

    accuracy                           0.88    160593
   macro avg       0.54      0.83      0.61    160593
weighted avg       0.94      0.88      0.90    160593

